In [2]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI PODCZAS WYSYŁANIA ##########################

FINAL_EVALUATION_MODE = False # Podczas sprawdzania ustawimy tą flagę na True.

In [3]:

######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

import os
import gdown
import pandas
import torch
import numpy as np
import torch.optim as optim
import torch.nn as nn

In [4]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def seed_everything(seed: int) -> None:
    """
    Ustawia ziarno (seed) dla odtwarzalności wyników w Pythonie, NumPy oraz PyTorch.

    Funkcja ustawia ziarno dla generatorów liczb losowych Pythonie, NumPy oraz PyTorch,
    a także konfiguruje PyTorch do pracy w trybie deterministycznym.

    Parametry:
        seed (int): Wartość ziarna do ustawienia.
    """
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [5]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

seed_everything(12345)

device = 'cuda'
assert torch.cuda.is_available(), "CUDA niedostępna!"


In [6]:
######################## NIE ZMIENIAJ TEJ KOMÓRKI ##########################

class CSVDataloader(torch.utils.data.DataLoader):
    """
    Klasa CSVDataloader służy do ładowania danych z plików CSV i zwracania ich w batchach.

    Przyjmuje:
        csv_file (str): Ścieżka do pliku CSV.
        batch_size (int): Rozmiar batcha.
        shuffle (bool): Czy tasować dane.
    """
    def __init__(self, csv_file, batch_size=128, shuffle=True):

        class CSVDataset(torch.utils.data.Dataset):
            """
            Klasa CSVDataset służy do przechowywania danych z plików CSV jako pojedyncze próbki.
            """
            def __init__(self, csv_file: str):
                data = pandas.read_csv(csv_file).values
                self.x = torch.tensor(data[:, :-1], dtype=torch.float32)  # Features
                self.y = torch.tensor(data[:, -1], dtype=torch.float32)  # Labels

            def __len__(self) -> int:
                return len(self.x)

            def __getitem__(self, idx: int) -> tuple:
                return self.x[idx].long(), self.y[idx]

        dataset = CSVDataset(csv_file)
        self.seq_len = dataset.x.shape[1]
        super().__init__(dataset, batch_size=batch_size, shuffle=shuffle)

In [7]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# Inicjalizacja treningowego zbioru danych
train_dataset_path = "train_dataset.csv"
val_dataset_path = "val_dataset.csv"

if not os.path.exists(train_dataset_path):
    url = "https://drive.google.com/uc?id=1INeYNpPA_YwojuQbMizlsFsERJ-PJX-E"
    gdown.download(url, train_dataset_path, quiet=True)

if not os.path.exists(val_dataset_path):
    url = "https://drive.google.com/uc?id=1oQcOMyDWVX0x76LOyp4TcFip1koRuodN"
    gdown.download(url, val_dataset_path, quiet=True)

dl = CSVDataloader("train_dataset.csv")
val_dl = CSVDataloader("val_dataset.csv")

In [8]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

def mse_criterium(
        estimations: torch.Tensor,
        answers: torch.Tensor
    ) -> torch.Tensor:
    """
    Oblicza wartość funkcji błędu średniokwadratowego (MSE) pomiędzy predykcjami a prawdziwymi wartościami.

    Parametry:
        estimations (torch.Tensor): Predykcje modelu.
        answers (torch.Tensor): Prawdziwe wartości.

    Zwraca:
        torch.Tensor: Wartość funkcji błędu średniokwadratowego.
    """
    return torch.mean((estimations.view(-1) - answers.view(-1)) ** 2)


def validate_model(
        model: torch.nn.Module,
        val_dl: torch.utils.data.DataLoader,
    ) -> float:
    """
    Waliduje model na zbiorze walidacyjnym. Zwraca uśrednioną wartość
    funkcji błędu średniokwadratowego dla wszystkich próbek.

    Parametry:
        model (torch.nn.Module): Model do oceny.
        val_dl (torch.utils.data.DataLoader): DataLoader z danymi walidacyjnymi.

    Zwraca:
        float: Uśredniona wartość funkcji błędu średniokwadr
    """
    model = model.eval().to(device)
    values = []
    for x, y in val_dl:
        x = x.to(device)
        y = y.to(device)
        y_pred = model(x)

        mse = mse_criterium(y_pred, y).cpu().item()
        values.append(mse)

    final_value = torch.tensor(values).mean().item()

    return final_value

def estimate_points(mse: float) -> int:
    """
    Funkcja wyznaczająca ilość punktów za zadanie na podstawie wartości funkcji błędu średniokwadratowego.

    Parametry:
        mse (float): Wartość funkcji błędu średniokwadratowego.

    Zwraca:
        int: Ilość punktów za zadanie (0 - 100).
    """
    points = max((100 * (64 - mse)) / 64, 0)
    return int(round(points))

In [9]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

class MLP(nn.Module):
    """
    Klasa reprezentująca model sieci MLP z czterema ukrytymi warstwami.

    Parametry:
        input_length (int): Długość wejścia sieci (długość sekwencji).
    """
    def __init__(self, input_length: int):
        super(MLP, self).__init__()
        neurons_num = [256, 128, 64, 32]

        self.fc_layers = nn.Sequential(
            nn.Linear(input_length, neurons_num[0]),
            nn.ReLU(),
            nn.Linear(neurons_num[0], neurons_num[1]),
            nn.ReLU(),
            nn.Linear(neurons_num[1], neurons_num[2]),
            nn.ReLU(),
            nn.Linear(neurons_num[2], neurons_num[3]),
            nn.ReLU(),
            nn.Linear(neurons_num[3], 1),
        )

        print("Liczba parametrów:", sum(p.numel() for p in self.parameters()))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Funkcja przyjmująca sekwencje danych i zwracająca predykcje ich wartości za pomocą sieci MLP.

        Parametry:
            x (torch.Tensor): Sekwencja danych.

        Zwraca:
            torch.Tensor: Predykcje wartości sekwencji.
        """
        x = x.float()
        x = self.fc_layers(x)
        return x

In [10]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
	model = MLP(dl.seq_len).to(device)

	optimizer = optim.Adam(model.parameters(), lr=0.005)
	criterion = nn.MSELoss()

	model.train()
	for batch in iter(dl): # pojedyncze przeiterowanie dl - 4000 batchy
		inputs, targets = batch
		inputs, targets = inputs.to(device).long(), targets.to(device).float().unsqueeze(1)

		optimizer.zero_grad()
		outputs = model(inputs)

		loss = criterion(outputs, targets)
		loss.backward()
		optimizer.step()

Liczba parametrów: 49665


In [11]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

# walidacja przykładowego rozwiązania
if not FINAL_EVALUATION_MODE:
    score = validate_model(model, val_dl)
    print(f"Błąd średniokwadratowy: {score:.2f}")

Błąd średniokwadratowy: 109.82


In [12]:
# przykładowy model, który możesz modyfikować
class YourModel(nn.Module):
    def __init__(self, input_length):
        super(YourModel, self).__init__()
        hidden_size = 33
        #hidden_size = 50
        num_layers = 2
        #num_layers = 3
        self.rnn = nn.LSTM(input_size=1, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)
        total_params = sum(p.numel() for p in self.parameters())
        #print(f"Liczba parametrów modelu: {total_params}")
    def forward(self, x):
        x = x.float()
        x, _ = self.rnn(x.unsqueeze(-1))
        x = self.fc(x[:, -1, :])
        return x
your_model = YourModel(dl.seq_len).to(device)
optimizer = optim.Adam(your_model.parameters(), lr=0.005)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1000, gamma=0.5)
criterion = nn.MSELoss()
best_loss = float("inf")
best_model_state = None
your_model.train()
total_loss = 0.0
for i, batch in enumerate(dl):
    inputs, targets = batch
    inputs, targets = inputs.to(device).float(), targets.to(device).float().unsqueeze(1)
    outputs = your_model(inputs)
    loss = criterion(outputs, targets)
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(your_model.parameters(), max_norm=1.0)
    optimizer.step()
    total_loss += loss.item()
    if (i + 1) % 500 == 0:
        avg_loss = total_loss / (i + 1)
    if (i + 1) % 1000 == 0:
        scheduler.step()
    if loss.item() < best_loss:
        best_loss = loss.item()
        best_model_state = your_model.state_dict()
if best_model_state:
    your_model.load_state_dict(best_model_state)

In [13]:
# ######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if not FINAL_EVALUATION_MODE:
    assert sum(p.numel() for p in your_model.parameters()) < 50000, "Model posiada za dużo parametrów"

    mse = validate_model(your_model, val_dl)
    score = estimate_points(mse)

    print(f"Błąd średniokwadratowy: {mse:.2f}")
    print(f"Estymowane punkty za zadanie: {score}")

Błąd średniokwadratowy: 6.91
Estymowane punkty za zadanie: 89


In [14]:
######################### NIE ZMIENIAJ TEJ KOMÓRKI ##########################

if FINAL_EVALUATION_MODE:
    import cloudpickle

    OUTPUT_PATH = "file_output"
    FUNCTION_FILENAME = "your_model.pkl"
    FUNCTION_OUTPUT_PATH = os.path.join(OUTPUT_PATH, FUNCTION_FILENAME)

    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    your_model = your_model.eval()

    with open(FUNCTION_OUTPUT_PATH, "wb") as f:
        cloudpickle.dump(your_model, f)